In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mpl_dates
from sklearn.linear_model import LinearRegression

# Visualizing and Analyzing Covid-19 Data for the Philippines

Disclaimer: Anything published in here are primarily based on data only and are done without any expertise in medicine or epidemiology. This is published for learning purposes only.

Data comes from the officially published data of the Department of Health in their data dump at https://drive.google.com/open?id=10VkiUA8x7TS2jkibhSZK1gmWxFM-EoZP and is subject to their Policy and Confidentiality Guidelines.

Load the data from a csv file in the data folder (updated every 7 PM Philippine Time) and examine the contents

In [5]:
df = pd.read_csv('datafiles\doh\DOH COVID Data Drop_ 20200504 - 05 Case Information.csv',
                 parse_dates=['DateRepConf', 'DateRecover', 'DateDied', 'DateRepRem'])

df.head(5)

,CaseCode,Age,AgeGroup,Sex,DateRepConf,DateRecover,DateDied,RemovalType,DateRepRem,Admitted,RegionRes,ProvRes,CityMunRes,RegionPSGC,ProvPSGC,CityMuniPSGC,HealthStatus,Quarantined
0,C100119,30.0,30 to 34,Male,2020-04-12,NaT,NaT,NaN,NaT,Yes,NCR,NCR,City of Parañaque,PH130000000,PH137600000,PH137604000,Asymptomatic,No
1,C100264,57.0,55 to 59,Male,2020-03-29,NaT,NaT,NaN,NaT,Yes,NCR,NCR,City of Mandaluyong,PH130000000,PH137400000,PH137401000,Mild,NaN
2,C100648,33.0,30 to 34,Female,2020-04-16,NaT,NaT,NaN,NaT,Yes,NCR,NCR,Quezon City,PH130000000,PH137400000,PH137404000,Mild,Yes
3,C100660,42.0,40 to 44,Female,2020-04-02,NaT,NaT,NaN,NaT,Yes,NCR,NCR,City of Pasig,PH130000000,PH137400000,PH137403000,Asymptomatic,Yes
4,C100776,42.0,40 to 44,Male,2020-04-01,NaT,NaT,NaN,NaT,Yes,4A,Laguna,Pila,PH040000000,PH043400000,PH043422000,Mild,NaN


In [6]:
n = 5
day_case = pd.DataFrame(df['DateRepConf'])
day_case = pd.DataFrame(day_case.groupby('DateRepConf')['DateRepConf'].count().reset_index(name='Confirmed'))
df_removed = df[df['RemovalType'].notna()]
df_removed = df_removed[['RemovalType', 'DateRepRem']]
df_removed = pd.DataFrame(df_removed.groupby(['DateRepRem', 'RemovalType']).size().reset_index(name='count'))
df_deaths = df_removed[df_removed['RemovalType'] == 'Died'].drop(columns='RemovalType').rename(columns={'count': 'Death'})
df_recovered = df_removed[df_removed['RemovalType'] == 'Recovered'].drop(columns='RemovalType').rename(columns={'count': 'Recovered'})
del df_removed
cases_time_series = pd.merge(day_case, df_deaths, how='left', left_on='DateRepConf', right_on='DateRepRem')
cases_time_series = pd.merge(cases_time_series, df_recovered, how='left', left_on='DateRepConf', right_on='DateRepRem')
cases_time_series = cases_time_series.drop(columns=['DateRepRem_x', 'DateRepRem_y']).set_index('DateRepConf').fillna(0)
del[day_case,df_deaths,df_recovered]
max_bars = cases_time_series.nlargest(n, ['Confirmed'])